In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from kaggle.competitions import twosigmanews

env = twosigmanews.make_env() 
(market_train_df, news_train_df) = env.get_training_data()
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))






In [ ]:
plt.rcParams['figure.figsize'] = (20.0, 10.0)

        **ABOUT THE DATA**
        The Data comes from the Kaggle competition titled ( Two Sigma: Using News to Predict Stock Movements ) in this website : https://www.kaggle.com/c/two-sigma-financial-news
        
        The data consists of two sets, The first is a table of market data and the second is a table of news information. 
        
        In this capstone project, I will explore the data, by first asking questions and then answering these question using visulization and statistics. 
   
      



News data provided by Thomson Reuters. Copyright ©, Thomson Reuters, 2017. All Rights Reserved. Use, duplication, or sale of this service, or data contained herein, except as described in the Competition Rules, is strictly prohibited.

## Data Overview

In [ ]:
print(f'{market_train_df.shape[0]} samples and {market_train_df.shape[1]} features in the training market dataset.')
    



In [ ]:
market_train_df.head()

In [ ]:
market_train_df.tail()

In [ ]:
market_train_df.dtypes

In [ ]:
print(f'{news_train_df.shape[0]} samples and {news_train_df.shape[1]} features in the training news dataset.')
    



In [ ]:
news_train_df.head()

In [ ]:
news_train_df.tail()

In [ ]:
news_train_df.dtypes

Having an overview of the data helps formulating the questions that I will try to explore and answer in the following steps.

The data is too big with many rows on both tables. however, the questions will only involve one company to minimize the data used in answering the questions






## First Question

what is  the distribution of the  daily change in price of a stock? let the Stock be of Apple Co.

In [ ]:
asset1Code = 'AAPL.O'
asset1_df = market_train_df[(market_train_df['assetCode'] == asset1Code) & (market_train_df['time'] < '2017-01-01')]

In [ ]:

sns.distplot(asset1_df['returnsClosePrevRaw1'], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
plt.axvline(asset1_df['returnsClosePrevRaw1'].mean(), color='r', linestyle='dashed', linewidth=2)
plt.xlabel('Apple returns at close time')
plt.ylabel('Frequency')
plt.title('Apple stock returns Frequency Distribution')

The histogram shows that the returns at close time for apple follows a normal distribution, this is what i expected but it is nice to see it. The mean of is actually around 0 (Does this mean stocks are a zero sum game ? ) 

## Second Question 
is there a correlation between the number of negative sentiment news articles and the fluctuations of the stocks of apple?


In [ ]:
asset2Code = 'Apple Inc'
asset2_df = news_train_df.loc[lambda df: df['assetName'] == asset2Code, :]

In [ ]:
asset2_df.head()

In [ ]:
## I will only keep the day-month-year in the time column and then merge both data sets on that

asset1_df['date'] = asset1_df['time'].dt.strftime(date_format='%Y-%m-%d')
asset2_df['date'] = asset2_df['time'].dt.strftime(date_format='%Y-%m-%d')
merged_df = pd.merge(asset1_df, asset2_df, on='date')

In [ ]:
merged_df.head()

In [ ]:
plot = pd.DataFrame()
plot = merged_df[['returnsClosePrevRaw1','returnsOpenPrevMktres1','sentimentNegative','sentimentNeutral','sentimentPositive']].copy()
plot.head()
sns.pairplot(plot)

In [ ]:
Third Question: Plot the word cloud for headlines of Apple Inc.

In [ ]:

# Python program to generate WordCloud 
  
# importing all necessery modules 
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
stop = set(STOPWORDS)
text = ' '.join(merged_df['headline'].str.lower().values)
wordcloud = WordCloud(max_font_size=None, stopwords=stop, background_color='white',
                      width=1200, height=1000).generate(text)
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud)
plt.title('Top words in headline of Apple')
plt.axis("off")
plt.show()
